In this assignment, you will use a pre-trained convnet to produce features for a classifier that can detect a single object type. This notebook has some code to help you get started. 

In [1]:
from imutils import paths
import pandas as pd
import requests
import cv2
import os
from os import listdir
from os.path import isfile, join
import os.path as osp
from tqdm import tqdm_notebook as tqdm
from google_images_download import google_images_download

img_folder = 'downloads'

In [2]:
def build_arguments(word):
    args = {}
    args['keywords'] = word
    args['limit'] = 100
    args['format'] = 'png'
    args['usage_rights'] = 'labeled-for-nocommercial-reuse'
    return args

response = google_images_download.googleimagesdownload()

### Run the following two cells

- change the positive and negative words
- running each cell downloads 100 images into a `downloads` folder with a subfolder of your word
- `word` can be any string or search criteria

In [3]:
# positive examples

word = 'red'
absolute_image_paths = response.download(build_arguments(word))


Item no.: 1 --> Item name = red
Evaluating...
Starting Download...
Completed Image ====> 1. 2000px-color_icon_red.svg.png
Completed Image ====> 2. shades_of_red.png
Completed Image ====> 3. 1024px-red.svg.png
Completed Image ====> 4. 1024px-color_icon_red.svg.png
Completed Image ====> 5. 1024px-disc_plain_red.svg.png
Completed Image ====> 6. 768px-disc_plain_red.svg.png
Completed Image ====> 7. digital-art-3193928_960_720.png
Completed Image ====> 8. 1024px-emblem-important-red.svg.png
Completed Image ====> 9. red_and_blue_800_%c3%97_600%2c_horizontal.png
Completed Image ====> 10. seal-1877460_960_720.png
Completed Image ====> 11. 1024px-red_copyright.svg.png
Completed Image ====> 12. 16134-illustration-of-a-red-heart-pv.png
Completed Image ====> 13. overlay?src0=https%3a%2f%2fi.vimeocdn.com%2fvideo%2f725468257_1280x720.jpg&src1=https%3a%2f%2ff.vimeocdn.com%2fimages_v6%2fshare%2fplay_icon_overlay.png.jpg
Completed Image ====> 14. background-line-pattern-colored-lines-pattern-red-15275

In [4]:
# negative examples

word = 'random'
absolute_image_paths = response.download(build_arguments(word))


Item no.: 1 --> Item name = random
Evaluating...
Starting Download...
Completed Image ====> 1. 1020px-34-gon-dissection-random.svg.png
Completed Image ====> 2. 1019px-30-gon-dissection-random.svg.png
Completed Image ====> 3. 2000px-28-gon-dissection-random.svg.png
Completed Image ====> 4. shapes-geometric-shades-collection-blue-random-1431991.png
Completed Image ====> 5. binary-1254484_960_720.png
Completed Image ====> 6. chart_random_pies02.png
Completed Image ====> 7. 1024px-20-gon-dissection-random.svg.png
Completed Image ====> 8. vqjxcfn6itf_clihq5bqtfd_p7du7cx0lzredzsbbok3wgnih3qgbo0n3knn2sxvay4io_fsho8ur65qkqslkhbggv8=s0.jpg
Completed Image ====> 9. 148ggoblin_by_jevi93-dcco06q.png
Completed Image ====> 10. 0-binary-ones-random-digital-1-numbers-zeroes-1254482.png
Completed Image ====> 11. jh9ki3c6qxnnjd1irulvz1afgxuwpysmchjmm8vpxhzjy6japlqt_a7slqdm2ntphe3w9-ekiny3cso_38xrnby2oma=s0.jpg
Completed Image ====> 12. tree-3515189_960_720.png
Completed Image ====> 13. 180stitch_by_jev

## 1.) Run the following cell

* This imports needed Keras libraries
* Then, it gets the trained VGG19 imagenet model
* Then, it prints out the names of all the layers in that model

In [17]:
import numpy as np
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model

base_model = VGG19(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
fc2
predictions


### 2.) Determine your output layer

- try `predictions` first

In [18]:
layer = 'flatten'
#layer = 'predictions'

model = Model(input=base_model.input, output=base_model.get_layer(layer).output)

C:\Users\arjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fl...)`
  after removing the cwd from sys.path.


### Run the following cell

In [19]:
def get_image(img_path, xs=224,ys=224):
    x = image.load_img(img_path, target_size=(xs, ys))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    return x

def get_img_features(model, img):
    img = preprocess_input(img)
    yhat = model.predict(img)
    return yhat

def get_image_features(word):
    files = [f for f in listdir(osp.join(img_folder, word))] # grab all of the images in the folder
    image_vectors = []
    for f in tqdm(files):
        img = get_image(osp.join(img_folder, word, f)) 
        x_feats = get_img_features(model, img).flatten() # get features for each image
        image_vectors.append(x_feats) 
    return np.array(image_vectors)

## 3.) Evaluate a classifier for your `word`

* Using the positive and negative output from `base_model`, train a classifier (it can be a linear classifier from scikit-learn, if you'd like, but I would recommend the Keras Dense network we built for the previous assignment). 
* You'll need to split your data into Train and Test (I would recommend using half of the data for training, half for testing; you may opt for downloading more positive and negative examples)

In [20]:
pos_images = get_image_features('red') # get positive image vectors
neg_images = get_image_features('random') # get negative image vectors

C:\Users\arjun\Anaconda3\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


### Prepare the data. Split to train/test sets

In [13]:
pos_images = pos_images[:100]
neg_images = neg_images[:100]

len(pos_images), len(neg_images)

(100, 100)

In [14]:
# split to train/test
num_test = 20
X_train = np.concatenate((pos_images[num_test:], neg_images[num_test:]),axis=0)
X_test = np.concatenate((pos_images[:num_test],  neg_images[:num_test]))

# labels? 1 = pos_images, 0 = neg_images
y_train = np.array([1] * (100 - num_test) + [0] * (100 - num_test))
y_test = np.array([1] * (num_test) + [0] * (num_test))

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((160, 25088), (160,), (40, 25088), (40,))

### Define model, train

In [15]:
from sklearn.linear_model import LogisticRegression

clfr = LogisticRegression()

clfr.fit(X_train, y_train)

C:\Users\arjun\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### Evaluate

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(clfr.predict(X_test), y_test)

0.9

## 4.) Try a Different `base_model` 

* Repeat steps 3 and 4 above, only this time use a [pre-trained model other than VGG19](https://keras.io/applications/)
* (Answer in a markdown cell): Which model+layer works the best for this data? Why do you think that is?

In [22]:
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

base_model = VGG16(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)

layer = 'predictions'

model = Model(input=base_model.input, output=base_model.get_layer(layer).output)

pos_images = get_image_features('red') # get positive image vectors
neg_images = get_image_features('random') # get negative image vectors

pos_images = pos_images[:100]
neg_images = neg_images[:100]

len(pos_images), len(neg_images)

# split to train/test
num_test = 20
X_train = np.concatenate((pos_images[num_test:], neg_images[num_test:]),axis=0)
X_test = np.concatenate((pos_images[:num_test],  neg_images[:num_test]))

# labels? 1 = pos_images, 0 = neg_images
y_train = np.array([1] * (100 - num_test) + [0] * (100 - num_test))
y_test = np.array([1] * (num_test) + [0] * (num_test))

X_train.shape, y_train.shape, X_test.shape, y_test.shape

from sklearn.linear_model import LogisticRegression

clfr = LogisticRegression()

clfr.fit(X_train, y_train)

accuracy_score(clfr.predict(X_test), y_test)

input_4
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
flatten
fc1
fc2
predictions


C:\Users\arjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  


C:\Users\arjun\Anaconda3\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


C:\Users\arjun\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.55

In [2]:
from client.api.notebook import Notebook
ok = Notebook('a7.ok')
import os
if not os.path.exists(os.path.join(os.environ.get("HOME"), ".config/ok/auth_refresh")):
    ok.auth(force=True)
else:
    ok.auth(inline=True)

Assignment: A7 Transfer Learning
OK, version v1.14.11


Open the following URL:

https://okpy.org/client/login/

After logging in, copy the code from the web page and paste it into the box.
Then press the "Enter" key on your keyboard.

Paste your code here: HtuMaZZ0wysL6ET5rnqq3XADy1dxh5
Successfully logged in as arjunshukla@u.boisestate.edu


In [ ]:
ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>